In [2]:
import mediapipe as mp
from mediapipe import tasks
from mediapipe.tasks.python import vision

import cv2
import numpy as np

from mediapipe.framework.formats import landmark_pb2
from mediapipe.python.solutions import drawing_utils, drawing_styles, pose

In [3]:
model_path = 'models/pose_landmarker_full.task'

In [4]:
def draw_landmarks_on_image(rgb_image, detection_result: vision.PoseLandmarkerResult):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for pose_landmarks in pose_landmarks_list:
        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend(
            [
                landmark_pb2.NormalizedLandmark(
                    x=landmark.x, y=landmark.y, z=landmark.z
                )
                for landmark in pose_landmarks
            ]
        )
        drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            pose.POSE_CONNECTIONS,
            drawing_styles.get_default_pose_landmarks_style(),
        )
    return annotated_image

[サンプル動画のダウンロードはこちらから](https://videos.pexels.com/video-files/3997798/3997798-uhd_2160_4096_25fps.mp4)

In [5]:
# Create a pose landmarker instance with the video mode:
options = vision.PoseLandmarkerOptions(
    base_options=tasks.BaseOptions(model_asset_path=model_path),
    running_mode=vision.RunningMode.VIDEO)

with vision.PoseLandmarker.create_from_options(options) as landmarker:
    cap = cv2.VideoCapture('videos/3997798-uhd_2160_4096_25fps.mp4')
    if not cap.isOpened():
        raise

    while True:
        ret, cv2_image = cap.read()
        if not ret:
            break

        # cv2.imshow('Sample Video', cv2_image)

        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2_image)

        # Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS
        # You’ll need it to calculate the timestamp for each frame.
        timestamp = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        # Perform pose landmarking on the provided single image.
        # The pose landmarker must be created with the video mode.
        pose_landmarker_result = landmarker.detect_for_video(mp_image, timestamp)

        annotated_image = draw_landmarks_on_image(cv2_image, pose_landmarker_result)
        cv2.imshow('Sample Video', annotated_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1718180631.764715  118237 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1718180631.852961  119498 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718180631.868122  119499 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/takato/Documents/chu-sama/PythonLab/.venv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [6]:
import time


with vision.PoseLandmarker.create_from_options(options) as landmarker:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise

    while True:
        ret, cv2_image = cap.read()
        if not ret:
            continue

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2_image)
        timestamp = int(time.time() * 1000)
        pose_landmarker_result = landmarker.detect_for_video(mp_image, timestamp)

        annotated_image = draw_landmarks_on_image(cv2_image, pose_landmarker_result)
        cv2.imshow('Sample Video', annotated_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1718180634.540282  118237 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3
W0000 00:00:1718180634.600588  119654 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718180634.605803  119654 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
